In [1]:
from collections import defaultdict, Counter
import datetime as dt
import emoji
from icecream import ic
from IPython.display import display_html 
import itertools
import matplotlib.pyplot as plt
import numpy as np
import os
import pandas as pd
import pickle
import random
import re
import regex as rex
import shutil
from string import punctuation
import time
from tqdm import tqdm

import nltk
from nltk.corpus import stopwords

import spacy

from sklearn.feature_extraction.text import TfidfTransformer, \
CountVectorizer, TfidfVectorizer
from sklearn.model_selection import train_test_split
from sklearn.pipeline import make_pipeline, Pipeline

import textacy.preprocessing as tprep
from textacy.extract import keyword_in_context

In [2]:
to_csv_flag = True
# Establish working directories for saving dataframes as CSV files.

'''Dir nav citation:
https://softhints.com/python-change-directory-parent/
'''
curr_dir = os.path.abspath(os.curdir)
print(curr_dir)
os.chdir("..")
up1_dir = os.path.abspath(os.curdir)
print(up1_dir)

C:\Users\acarr\Documents\GitHub\599_team_project\deliverables
C:\Users\acarr\Documents\GitHub\599_team_project


In [3]:
# Get current date/time to append to file name string.

today = dt.datetime.today()
today= str(today)
today = today.replace(':', '-').replace('.', '').replace(' ', '_')
print(today)
print(type(today))

2023-07-28_12-11-33337653
<class 'str'>


In [4]:
# change `data_large_location` to the location of the folder on your machine.
data_large_location = 'data_large'
ref_docs_location = 'ref_docs'

file_in_name01 = 'news-04.csv'

file_in_path01 = os.path.join(up1_dir, data_large_location, file_in_name01)

print(f'CSV file in 1 path: {file_in_path01}')

CSV file in 1 path: C:\Users\acarr\Documents\GitHub\599_team_project\data_large\news-04.csv


### Review dataframe

Read in data from CSV, check resulting dataframe shape, and display first several records.

In [5]:
slct_tbl_full_df01 = pd.read_csv(file_in_path01)
print(f'Dataframe shape: {slct_tbl_full_df01.shape}')
display(slct_tbl_full_df01.head())

Dataframe shape: (36405, 8)


,text_id,sentiment_vader,sentiment_vader_compound,sentiment_vader_cat,sentiment_bert_prob,sentiment_bert_cat,sentiment_bert,embeddings
0,2,"{'neg': 0.041, 'neu': 0.888, 'pos': 0.072, 'co...",0.8658,positive,0.009480,negative,-0.981040,"tensor([-7.5552e-02, 5.0567e-03, -4.7665e-02,..."
1,3,"{'neg': 0.043, 'neu': 0.85, 'pos': 0.107, 'com...",0.9769,positive,0.981083,positive,0.962166,"tensor([ 4.0231e-02, -1.2047e-02, 7.2875e-02,..."
2,5,"{'neg': 0.02, 'neu': 0.817, 'pos': 0.163, 'com...",0.9900,positive,0.013669,negative,-0.972662,"tensor([-1.1686e-01, -1.0624e-01, -7.8918e-03,..."
3,6,"{'neg': 0.016, 'neu': 0.888, 'pos': 0.096, 'co...",0.9788,positive,0.993403,positive,0.986807,"tensor([-9.5328e-03, 4.5495e-03, -9.8373e-02,..."
4,7,"{'neg': 0.061, 'neu': 0.853, 'pos': 0.085, 'co...",0.6697,positive,0.910206,positive,0.820413,"tensor([-2.5785e-02, 7.9342e-02, -2.9590e-02,..."


In [6]:
# Set path to write CSV file to.

file_out_name01 = f'data_sa_w_sw_half1_{today}.csv'
file_out_name02 = f'data_sa_w_sw_half2_{today}.csv'

file_out_path01 = os.path.join(up1_dir, file_out_name01)
file_out_path02 = os.path.join(up1_dir, file_out_name02)

print(f'CSV file out 1 path: {file_out_path01}')
print(f'CSV file out 2 path: {file_out_path02}')

export_col_names_lst = ['text_id',
                        'sentiment_bert',
                       ]

# Write pandas dataframe to CSV; save locally.

len_half = int(round(len(slct_tbl_full_df01)/2,0))
print(len_half)

if to_csv_flag == False:
    pass
else:
    slct_tbl_full_df01[export_col_names_lst][:len_half]\
    .to_csv(file_out_path01, index=False)
    slct_tbl_full_df01[export_col_names_lst][len_half:]\
    .to_csv(file_out_path02, index=False)

print(slct_tbl_full_df01.shape)
display(slct_tbl_full_df01.sample(5))

CSV file out 1 path: C:\Users\acarr\Documents\GitHub\599_team_project\data_sa_w_sw_half1_2023-07-28_12-11-33337653.csv
CSV file out 2 path: C:\Users\acarr\Documents\GitHub\599_team_project\data_sa_w_sw_half2_2023-07-28_12-11-33337653.csv
18202
(36405, 8)


,text_id,sentiment_vader,sentiment_vader_compound,sentiment_vader_cat,sentiment_bert_prob,sentiment_bert_cat,sentiment_bert,embeddings
24961,27157,"{'neg': 0.073, 'neu': 0.854, 'pos': 0.073, 'co...",0.9562,positive,0.003012,negative,-0.993975,"tensor([-8.2699e-02, -4.4747e-03, 7.8296e-02,..."
11073,12720,"{'neg': 0.062, 'neu': 0.889, 'pos': 0.048, 'co...",-0.7906,negative,0.002056,negative,-0.995889,"tensor([-6.5579e-02, 2.4717e-02, 1.1945e-01,..."
3547,4865,"{'neg': 0.128, 'neu': 0.82, 'pos': 0.053, 'com...",-0.9869,negative,0.003218,negative,-0.993564,"tensor([ 8.5340e-03, 1.2915e-01, -4.0248e-02,..."
1822,2705,"{'neg': 0.033, 'neu': 0.822, 'pos': 0.145, 'co...",0.9938,positive,0.954599,positive,0.909198,"tensor([-5.0319e-02, 2.3046e-02, 1.7523e-03,..."
11706,13461,"{'neg': 0.034, 'neu': 0.824, 'pos': 0.142, 'co...",0.9923,positive,0.151442,negative,-0.697115,"tensor([-3.2121e-02, 8.1591e-03, -4.0179e-02,..."
